https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ja

https://blog.tensorflow.org/2021/08/pose-estimation-and-classification-on-edge-devices-with-MoveNet-and-TensorFlow-Lite.html?hl=ja&_gl=1*xgz6jk*_ga*MTYwMDI2MzYzMC4xNzA0MTcwODE4*_ga_W0YLR4190T*MTcwNjUxOTU5NC41LjEuMTcwNjUyMDE0OC4wLjAuMA..

TFlite MOvenet RasPi sample code

https://github.com/tensorflow/examples/tree/master/lite/examples/pose_estimation/raspberry_pi#run-the-pose-classification-sample

Install dependencies. 
Run this script to install the Python dependencies, and download the TFLite models. sh setup.sh

for the error following:
ERROR: Could not find a version that satisfies the requirement tflite-runtime>=2.7.0 (from versions: none)
https://temcee.hatenablog.com/entry/tensorflow_install_error

In [2]:
! sh setup.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 12.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [259 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_machine != "aarch64" and platform_machine != "arm64"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.6" and sys_platform == "linux" and platform_machine == "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_machine != "aarch64" and platform_machine != "arm64"' don't match your environment
      Ignoring numpy: mark

-e Downloaded files are in ./


Mac OpenCV install https://qiita.com/makki_maki04/items/f62c5e4e68709d9b3b89
Mac Git install https://qiita.com/suke_masa/items/4bed855628f7414293f8
Homebrew install https://brew.sh/
Mac Tensorflow install https://www.tensorflow.org/install/pip?hl=ja#macos

In [ ]:
python3 pose_estimation.py

In [ ]:
! python3 pose_estimation.py

上記部分はmodule not found errorがでていたがカーネル再起動で通った

In [1]:
"""Main script to run pose classification and pose estimation."""
import argparse
import logging
import sys
import time

import cv2

import utils
from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

In [2]:
def myRun(camera_id: int, width: int, height: int) -> None:
    model = 'movenet_multipose'
    pose_detector = MoveNetMultiPose(model,'bounding_box',512)

    # Variables to calculate FPS
    counter, fps = 0, 0
    start_time = time.time()

    # Start capturing video input from the camera
    cap = cv2.VideoCapture(camera_id)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

    # Visualization parameters
    row_size = 20  # pixels
    left_margin = 24  # pixels
    text_color = (0, 0, 255)  # red
    font_size = 1
    font_thickness = 1
    classification_results_to_show = 3
    fps_avg_frame_count = 10
    keypoint_detection_threshold_for_classifier = 0.1
    classifier = None
    
    #key = cv2.waitKey(0)
    #cv2.startWindowThread()

    # Continuously capture images from the camera and run inference
    while cap.isOpened():
        success, image = cap.read()
        if not success:
          sys.exit(
              'ERROR: Unable to read from webcam. Please verify your webcam settings.'
          )

        counter += 1
        image = cv2.flip(image, 1)
        persons = pose_detector.detect(image)

        # Draw keypoints and edges on input image
        image = utils.visualize(image, persons)

        # Calculate the FPS
        if counter % fps_avg_frame_count == 0:
          end_time = time.time()
          fps = fps_avg_frame_count / (end_time - start_time)
          start_time = time.time()

        # Show the FPS
        fps_text = 'FPS = ' + str(int(fps))
        text_location = (left_margin, row_size)
        cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    font_size, text_color, font_thickness)

        # Stop the program if the q key is pressed.
        if cv2.waitKey(1) == ord('q'):
          break
        cv2.imshow(model, image)
        '''
        key = cv2.waitKey(0)

        if key == ord('q'):            #qを押した時の処理
            cv2.waitKey(1)
            cv2.destroyAllWindows()  
            cap.release()
            cv2.waitKey(1)
            '''
            
    cv2.destroyAllWindows()  
    cap.release()

In [3]:
myRun(0, 1280, 720)

SystemExit: ERROR: Unable to read from webcam. Please verify your webcam settings.

/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


qを押すとクラッシュしないように閉じる版

In [ ]:
import cv2
import numpy as np
import serial
import time

import argparse
import logging
import sys
import utils

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
camera_id = 1

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

image = np.zeros((720, 1280,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
row_size = 20  # pixels
left_margin = 24  # pixels
text_color = (0, 0, 255)  # red
font_size = 1
font_thickness = 1
classification_results_to_show = 3
fps_avg_frame_count = 10
keypoint_detection_threshold_for_classifier = 0.1
classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)

    # Draw keypoints and edges on input image
    image = utils.visualize(image, persons)
    
    #canvas = np.zeros((720, 1280,3), np.uint8)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)
    time.sleep(0.01)
    
    cv2.imshow(model, image)
    
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break

2024-02-08 11:07:01.211 python[21450:951668] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


課題：opencvで画面を出して、配列５m分を用意して表示する

In [2]:
import cv2
import numpy as np
import serial
import time
import logging
import sys
import utils

from typing import List, Tuple
from data import Person

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

class Body:
    id = 0
    bounding_box_start_point = (10,10)
    bounding_box_end_point = (20,20)
    

#1 do inference 
#2 convert the results to the list of Body objects
def process_bodies(
    list_persons: List[Person],
    keypoint_threshold: float = 0.8,
    instance_threshold: float = 0.8,
)-> List[Body]:
    
    bodies = [];
    for person in list_persons:
        if person.score < instance_threshold:
            continue
        keypoints = person.keypoints
        bounding_box = person.bounding_box
        
        # Draw bounding_box with multipose
        if bounding_box is not None:
            body = Body()
            body.bounding_box_start_point  =  bounding_box.start_point
            body.bounding_box_end_point = bounding_box.end_point
            body.id = person.id
            bodies.append(body)
        
    return bodies

def draw_bodies(
    image: np.ndarray,
    list_bodies: List[Body],
) -> np.ndarray:
    print("ab Persons:",len(list_bodies))
    for body in list_bodies:
        print("startp:",body.bounding_box_start_point[0],body.bounding_box_start_point[1])
        cv2.rectangle(image,body.bounding_box_start_point,body.bounding_box_end_point, (255, 255, 255), 2)
        id_text = 'id = ' + str(body.id)
        cv2.putText(image, id_text, body.bounding_box_start_point, cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255), 1)
    return image

def draw_bodies_line(
    image: np.ndarray,
    list_bodies: List[Body],
) -> np.ndarray:
    print("abl Persons:",len(list_bodies))
    temp_height = int(height/2)
    for body in list_bodies:
        cv2.rectangle(image,(body.bounding_box_start_point[0],temp_height),(body.bounding_box_end_point[0],temp_height), (255, 255, 255), 2)
        id_text = 'id = ' + str(body.id)
        cv2.putText(image, id_text, (body.bounding_box_start_point[0],temp_height), cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255), 1)
    return image
    

def draw_persons(
    image: np.ndarray,
    list_persons: List[Person],
    keypoint_color: Tuple[int, ...] = None,
    keypoint_threshold: float = 0.8,
    instance_threshold: float = 0.8,
) -> np.ndarray:
    for person in list_persons:
        if person.score < instance_threshold:
            continue
        keypoints = person.keypoints
        bounding_box = person.bounding_box
        
        # Draw bounding_box with multipose
        if bounding_box is not None:
            start_point = (bounding_box.start_point[0],int(height/2))
            end_point = (bounding_box.end_point[0],int(height/2))
            cv2.rectangle(image, start_point, end_point, (255, 255, 255), 2)
            # Draw id text when tracker is enabled for MoveNet MultiPose model.
            # (id = None when using single pose model or when tracker is None)
            if person.id:
                id_text = 'id = ' + str(person.id)
                cv2.putText(image, id_text, start_point, cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255), 1)
    return image



model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
camera_id = 1

led_length = 150;  #5m 150
leds = [0] * led_length

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

image = np.zeros((720, 1280,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
row_size = 20  # pixels
left_margin = 24  # pixels
text_color = (0, 0, 255)  # red
font_size = 1
font_thickness = 1
classification_results_to_show = 3
fps_avg_frame_count = 10
keypoint_detection_threshold_for_classifier = 0.1
classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)

    # Draw keypoints and edges on input image
    #image = utils.visualize(image, persons,keypoint_threshold=0.2,instance_threshold=0.2)
    bodies = process_bodies(persons,keypoint_threshold=0.2,instance_threshold=0.2)
    image = draw_bodies(image, bodies)
    #image = draw_bodies_line(image, bodies)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)
    
    cv2.imshow(model, image)
    time.sleep(0.02)
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break
        

    

ab Persons: 0
ab Persons: 1
startp: 562 493
ab Persons: 1
startp: 548 498
ab Persons: 0
ab Persons: 1
startp: 563 518
ab Persons: 1
startp: 553 497
ab Persons: 1
startp: 565 495
ab Persons: 1
startp: 564 494
ab Persons: 1
startp: 567 495
ab Persons: 1
startp: 570 498
ab Persons: 1
startp: 562 512
ab Persons: 1
startp: 558 513
ab Persons: 1
startp: 558 512
ab Persons: 1
startp: 560 513
ab Persons: 1
startp: 560 512
ab Persons: 1
startp: 553 513
ab Persons: 1
startp: 561 519
ab Persons: 1
startp: 556 522
ab Persons: 1
startp: 555 502
ab Persons: 1
startp: 558 501
ab Persons: 1
startp: 570 498
ab Persons: 1
startp: 570 498
ab Persons: 1
startp: 561 503
ab Persons: 1
startp: 566 518
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 1
startp: 563 515
ab Persons: 1
startp: 554 524
ab Persons: 1
startp: 553 501
ab Persons: 1
startp: 563 501
ab Persons: 1
startp: 565 494
ab Persons: 1
startp: 560 494
ab Persons: 1
startp: 559 495
ab P

ab Persons: 1
startp: 560 523
ab Persons: 1
startp: 562 523
ab Persons: 1
startp: 558 526
ab Persons: 1
startp: 554 523
ab Persons: 1
startp: 555 523
ab Persons: 1
startp: 557 518
ab Persons: 1
startp: 564 519
ab Persons: 0
ab Persons: 1
startp: 560 517
ab Persons: 1
startp: 561 515
ab Persons: 1
startp: 561 513
ab Persons: 1
startp: 563 513
ab Persons: 1
startp: 559 517
ab Persons: 1
startp: 561 517
ab Persons: 1
startp: 560 518
ab Persons: 1
startp: 560 518
ab Persons: 1
startp: 562 518
ab Persons: 1
startp: 559 518
ab Persons: 1
startp: 579 518
ab Persons: 1
startp: 564 508
ab Persons: 2
startp: 541 506
startp: 337 503
ab Persons: 1
startp: 500 505
ab Persons: 1
startp: 484 504
ab Persons: 1
startp: 506 506
ab Persons: 1
startp: 535 508
ab Persons: 1
startp: 563 511
ab Persons: 1
startp: 492 512
ab Persons: 1
startp: 524 523
ab Persons: 1
startp: 553 520
ab Persons: 1
startp: 559 521
ab Persons: 1
startp: 560 520
ab Persons: 0
ab Persons: 1
startp: 566 521
ab Persons: 1
startp: 562 

ab Persons: 1
startp: 564 515
ab Persons: 1
startp: 561 516
ab Persons: 1
startp: 562 516
ab Persons: 1
startp: 571 518
ab Persons: 1
startp: 564 516
ab Persons: 1
startp: 563 515
ab Persons: 1
startp: 579 514
ab Persons: 1
startp: 579 517
ab Persons: 1
startp: 568 518
ab Persons: 1
startp: 566 518
ab Persons: 1
startp: 568 518
ab Persons: 1
startp: 567 518
ab Persons: 1
startp: 568 519
ab Persons: 1
startp: 568 519
ab Persons: 1
startp: 565 519
ab Persons: 1
startp: 567 521
ab Persons: 1
startp: 566 520
ab Persons: 1
startp: 568 519
ab Persons: 1
startp: 567 519
ab Persons: 1
startp: 570 520
ab Persons: 1
startp: 570 520
ab Persons: 1
startp: 567 521
ab Persons: 1
startp: 565 521
ab Persons: 1
startp: 564 521
ab Persons: 1
startp: 566 520
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 0
ab Persons: 1
startp: 567 527
ab Persons: 1
startp: 559 523
ab Persons: 1
startp: 565 522
ab Persons: 1
startp: 564 522
ab Persons: 1
startp: 558 518
ab Persons: 1
st

課題：Serialに繋いで、バイト配列を送る